In [53]:
import re

def clean035(q):
    #remove WaOLN"(WaOLN)rye0053656", with and without ""
    q=re.sub('\"\(WaOLN\)rye[01-9]+\"','',q)
    q=re.sub('\(WaOLN\)rye[0-9]+','',q)
    q=re.sub('\(DNLM\)[0-9\(\)a-z]*','',q,flags=re.IGNORECASE)
    q=re.sub('\.b[0-9]*','',q)
    q=re.sub('LCMARC[a-z-0-9]*','',q,flags=re.IGNORECASE)
    q=re.sub('A[a-z-0-9]*','',q,flags=re.IGNORECASE)
    q=re.sub('U[a-z-0-9]*','',q,flags=re.IGNORECASE)
    q=re.sub('\(SIRSI\)[0-9a-z\-]*','',q,flags=re.IGNORECASE)
    q=re.sub('[a-z][a-z][a-z]-[0-9][0-9][0-9][0-9]','',q,flags=re.IGNORECASE)
    q=re.sub('[\"]*\(CaOTULAS\)[0-9a-z]*[\"]*','',q,flags=re.IGNORECASE)
    q=re.sub('[\"]*cn\s[0-9a-z]*[\"]*','',q,flags=re.IGNORECASE)
    q=re.sub('\sn\s','',q,flags=re.IGNORECASE)
    q=re.sub('\/[\/a-z0-9]*','',q,flags=re.IGNORECASE)
    q=re.sub('\(MN\)','',q,flags=re.IGNORECASE)
    q=re.sub('CL','',q,flags=re.IGNORECASE)
    q=re.sub('DLC[\sa-z0-9]','',q,flags=re.IGNORECASE)
    q=re.sub('sn[\s0-9]*','',q)
    q=re.sub('\,C,','',q)
    q=re.sub('CNMRCT','',q)
    q=re.sub('TRENT','',q)
    q=re.sub('v[\s0-9a-z\(\)]*','',q,flags=re.IGNORECASE)
    q=re.sub('w[\s0-9a-z\(\)]*','',q,flags=re.IGNORECASE)
    q=re.sub('s[\s0-9a-z\(\)]*','',q,flags=re.IGNORECASE)
    q=re.sub('z[\s0-9a-z\(\)]*','',q,flags=re.IGNORECASE)
    q=re.sub('r[\s0-9a-z\(\)]*','',q,flags=re.IGNORECASE)
    q=re.sub('y[\s0-9a-z\(\)]*','',q,flags=re.IGNORECASE)
    q=re.sub('^c[\s0-9a-z]*','',q,flags=re.IGNORECASE)
    q=re.sub('r[\s0-9a-z\(\)]*','',q,flags=re.IGNORECASE)
    q=re.sub('OO','',q)
    q=re.sub('M[\s]*','',q)
    q=re.sub('NYPL','',q)
    q=re.sub('orig[\sa-z0-9]*','',q)
    q=re.sub('LI','',q)
    q=re.sub('ce\s[0-9]*','',q)
    q=re.sub('\"','',q)
    q=re.sub(',,','',q)
    
#remove unnecessary comma at the beginnin
    q=re.sub('^,','',q)
#remove unnecessary comma at the end 
    q=re.sub(',\Z','',q)
    return q

In [190]:
# monographs 

print('-'*100)
print('MONOGRAPHS')
print('-'*100)
#files in raw folder
import pandas as pd
import re

df1=pd.read_csv('raw/print_mono_hathitrust_1.txt', sep='\t')
df2=pd.read_csv('raw/print_mono_hathitrust_2.txt', sep='\t')
df3=pd.read_csv('raw/print_mono_hathitrust_3.txt', sep='\t')

df1=df1[['35','RECORD #(BIBLIO)','BCODE3','BLIO)","VOLUME"','GOVT DOC #']]
df2=df2[['35','RECORD #(BIBLIO)','BCODE3','BLIO)","VOLUME"','GOVT DOC #']]
df3=df3[['35','RECORD #(BIBLIO)','BCODE3','BLIO)","VOLUME"','GOVT DOC #']]

frames = [df1, df2, df3]

df = pd.concat(frames)



print('-'*50)
print('Before cleanup')
print('-'*50)
df.info()
# run this only for serials 

df=df[['35','RECORD #(BIBLIO)','BCODE3','BLIO)","VOLUME"','GOVT DOC #']]


df['35']=df['35'].astype('str')
df['35']=df['35'].apply(clean035)
print('-'*50)
print('Before removing the record without 035')
print('-'*50)
df.info()


#CODE BCODE 3 according to instruction from Hathitrust
df['BCODE3']=df['BCODE3'].replace({'-':'CH','w':'WD'}) 

df.columns=['OCLC#',"Partner's Local System ID","Holding Status","Numeration and Chronology","Gov Doc Indicator"]

print("-"*50)
print("Number Based on Holding Status")
print("-"*50)

print(df.groupby(['Holding Status'])["Partner's Local System ID"].count())

# only need CH and WD 
print("-"*50)
print("Number with only Current Holdings and Withdrawn")
print("-"*50)
df=pd.concat([df[df['Holding Status']=='CH'],df[df['Holding Status']=='WD']])
print(df.groupby(['Holding Status'])["Partner's Local System ID"].count())


# set np.nan
import numpy as np
df.fillna(value=pd.np.nan, inplace=True)
df=df.replace('', np.nan) 
df.replace('nan',np.nan,inplace=True)


df=df[df['OCLC#']!='']
print('-'*50)
print("After removing the record without 035")
print('-'*50)
df.info()

df.info()
print("-"*50)
print("After final cleanup, remove 'n', remove the one with length less than 6 ")
print('-'*50)
#df=df[df['OCLC#']!='n']

df=df[df['OCLC#'].str.len()>6]
df.info()

--------------------------------------------------
Before cleanup
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 468932 entries, 0 to 68931
Data columns (total 5 columns):
35                  261911 non-null object
RECORD #(BIBLIO)    468932 non-null object
BCODE3              468932 non-null object
BLIO)","VOLUME"     460552 non-null object
GOVT DOC #          6814 non-null object
dtypes: object(5)
memory usage: 21.5+ MB
--------------------------------------------------
Before removing the record without 035
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 468932 entries, 0 to 68931
Data columns (total 5 columns):
35                  468932 non-null object
RECORD #(BIBLIO)    468932 non-null object
BCODE3              468932 non-null object
BLIO)","VOLUME"     460552 non-null object
GOVT DOC #          6814 non-null object
dtypes: object(5)
memory usage: 21.5+ MB
----------------

In [195]:
#serials

print('-'*100)
print('SERIALS')
print('-'*100)

# regex 
import pandas as pd
import re

df=pd.read_excel('data/print_bib_lvl_s.xls')
df.head()

print('-'*50)
print('Before cleanup')
print('-'*50)
df.info()
# run this only for serials 

df=df[['35','RECORD #(BIBLIO)','BCODE3','Volume','ISN','GOVT DOC #']]


df['35']=df['35'].astype('str')
df['35']=df['35'].apply(clean035)
print('-'*50)
print('Before removing the record without 035')
print('-'*50)
df.info()



#CODE BCODE 3 according to instruction from Hathitrust
df['BCODE3']=df['BCODE3'].replace({'-':'CH','w':'WD'}) 

df.columns=['OCLC#',"Partner's Local System ID","Holding Status","Numeration and Chronology","ISSN","Gov Doc Indicator"]

print("-"*50)
print("Number with Based on Status")
print("-"*50)

print(df.groupby(['Holding Status'])["Partner's Local System ID"].count())

# only need CH and WD 
print("-"*50)
print("Number with only Current Holdings and Withdrawn")
print("-"*50)
df=pd.concat([df[df['Holding Status']=='CH'],df[df['Holding Status']=='WD']])
print(df.groupby(['Holding Status'])["Partner's Local System ID"].count())


# set np.nan
import numpy as np
df.fillna(value=pd.np.nan, inplace=True)
df=df.replace('', np.nan) 
df.replace('nan',np.nan,inplace=True)


df=df[df['OCLC#']!='']
print('-'*50)
print("After removing the record without 035")
print('-'*50)
df.info()

df.info()
print("-"*50)
print("After final cleanup, remove 'n'")
print('-'*50)
df=df[df['OCLC#']!='n']
df.info()

----------------------------------------------------------------------------------------------------
SERIALS
----------------------------------------------------------------------------------------------------
--------------------------------------------------
Before cleanup
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7819 entries, 0 to 7818
Data columns (total 8 columns):
Column1             7819 non-null int64
35                  6678 non-null object
RECORD #(BIBLIO)    7819 non-null object
BCODE3              7819 non-null object
ISN                 5675 non-null object
GOVT DOC #          393 non-null object
rec_no              7819 non-null object
Volume              7819 non-null object
dtypes: int64(1), object(7)
memory usage: 488.8+ KB
--------------------------------------------------
Before removing the record without 035
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex

In [158]:
#Use this as a window to inspect the data 
df.sample(20)

,OCLC#,Partner's Local System ID,Holding Status,Numeration and Chronology,ISSN,Gov Doc Indicator
4638,05018346,b12108765,CH,"""1978"",""1981"",""1985"",""1987"",""1988"",""1992""",0706-7143,NaN
7541,n,b20247266,CH,"""v. 1""",NaN,NaN
5339,"90657474 ,(OCoLC)20765089",b12185243,CH,"""1989-90"",""1990-91"",""1991-92"",""1992-93"",""1993-...",NaN,NaN
1025,(OCoLC)2807178,b12234394,CH,"""v.21 (1977) - v.59:no.11 (November 2015)"",""v....",0145-9406,NaN
1219,(OCoLC)1553156,b12237498,CH,"""v.1:no.1 (Mar. 1968) - v.38:no.4 (Dec. 2005)""",0008-4239,NaN
5075,(OCoLC)18630667,b12106082,CH,"""1986"",""1989"",""1990""",0839-8917,NaN
4082,85335338,b13323441,CH,"""1938"",""1939"",""1939"",""1940"",""1941"",""1942"",""194...",NaN,NaN
2038,(OCoLC)3316919,b12369639,CH,"""v.26:no.1 (1979) -"",""v.32:no.1 (1985) - v.37:...",0049-3155,NaN
7210,n,b19247862,CH,"""2007-08""",1915-6731,NaN
7687,"1174657,31my83et[]",b12392947,CH,"""v. 14 No. 2"",""v. 16 No. 1"",""v. 16 No. 6""",NaN,NaN


In [166]:
#code to test regex (ISSN)
test ="9780345811820 (software),0345811828, [$128.95] New York"
print(re.sub(r'\([a-z\s.0-9\[\]]*\)','',test,flags=re.IGNORECASE))
print(re.sub(r'\$[0-9\*\.[0-9]*','',test))


9780345811820 ,0345811828, [$128.95] New York
9780345811820 (software),0345811828, [] New York


In [41]:
# function to clean up ISBN, turns out I don't need this because the serials ISSN are pretty clean, for monographs, ISBN are not required 

def ISSN(i):
    
#remove price ($100.00)
        i=re.sub('\$[0-9]*\.[0-9]*','',i)
#remove (softcover) (pbk.) (New York)
        i=re.sub('\([a-z\s.\[\]]*\)','',i,flags=re.IGNORECASE)
#remove pbk. New York
        i=re.sub('[a-z\s.]*','',i,flags=re.IGNORECASE)
#remove ((pbk.))
        i=re.sub('\(\([a-z]*\s[a-z]*\.\)\)','',i,flags=re.IGNORECASE)
#remove unwanted commas, 
        i=re.sub(r',,','',i,flags=re.IGNORECASE)
    
#remove </td> </tr>
        i=re.sub(r'<[//a-z]>','',i,flags=re.IGNORECASE)

#remove (:)
        i=re.sub('\([:]\)','',i)
# remove unwanted comma at the end 
        i=re.sub(',\Z','',i)

        return i 

In [168]:
df['ISN']=df.ISN.astype('str')
df['ISN']=df.ISN.apply(ISSN)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [121]:

#Test code for 035
test=',0388251,"(WaOLN)rye001,sn 3582225,on258359325"'
#test='n,,"",""'
test=re.sub('\"\(WaOLN\)rye[01-9]+\"','',test)
test=re.sub('^,','',test) 
test=re.sub(',\Z','',test)
test=re.sub('n/s','',test)
test=re.sub('\"','',test)
test=re.sub(',,','',test)
test=re.sub('sn[\s0-9]*','',test)
print(test)
#print(re.sub(r'\z,\z','',test))

0388251,(WaOLN)rye001,,on258359325


In [115]:
# don't need the lambda function anymore 
#035
#mono.ISN=mono.ISN.apply(lambda x: re.sub('\$[0-9]*\.[0-9]*','',x))
#mono.ISN=mono.ISN.apply(lambda x: re.sub('\([a-z\s.]*\)','',x,flags=re.IGNORECASE))
#mono.ISN=mono.ISN.apply(lambda x: re.sub('[a-z\s.]*','',x,flags=re.IGNORECASE))
#mono.ISN=mono.ISN.apply(lambda x: re.sub('\([:]\)','',x))
#mono['35']=mono['35'].apply(lambda x:re.sub(',\Z','',x))

#035
#mono['35']=mono['35'].astype('str')
#mono['35']=mono['35'].apply(lambda x:re.sub('\"\(WaOLN\)rye[01-9]+\"','',x))

In [191]:
#Last part of the code. Just change the name when processing serials 
df.to_csv('mono20200506.txt', sep='\t', index=False)